## Imports

In [ ]:
import cv2
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
from tensorflow.keras import layers
import keras

## Utility functions

### Callbacks

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                                  patience=3)

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                 factor=0.2,
                                                 patience=2,
                                                 verbose=1,
                                                 min_lr=1e-7)

### Data Genaration

In [ ]:
def generate_data(train_dir = "/content/drive/MyDrive/larynx_dataset/train",
                  test_dir = "/content/drive/MyDrive/larynx_dataset/test",
                  valid_dir = "/content/drive/MyDrive/larynx_dataset/validation"):
  datagen = ImageDataGenerator(rescale=1/255.)

  train_data = datagen.flow_from_directory(directory=train_dir,
                                                batch_size=32,
                                                target_size=(224, 224),
                                                class_mode="binary",
                                                seed=42)
  test_data = datagen.flow_from_directory(directory=test_dir,
                                                batch_size=32,
                                                target_size=(224, 224),
                                                class_mode="binary",
                                                seed=42)
  validation_data = datagen.flow_from_directory(directory=valid_dir,
                                                batch_size=32,
                                                target_size=(224, 224),
                                                class_mode="binary",
                                                seed=42)

  return train_data, test_data, validation_data

### Models Build

#### Pre-trained models

In [ ]:
def create_model(model_url, num_classes=1, IMAGE_SHAPE=(224, 224), BATCH_SIZE=32):
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False,
                                           name="feature_extraction_layer",
                                           input_shape=IMAGE_SHAPE+(3, ))
  model = tf.keras.Sequential([
      feature_extractor_layer,
      layers.Dense(num_classes, activation="sigmoid", name="output_layer")
  ])

  return model

#### Ensemble

In [ ]:
def ensemble(models, model_input):
  Models_output = [model(model_input) for model in models]
  AVG = layers.average(Models_output)

  model_ensemble = keras.models.Model(inputs=model_input, outputs=AVG, name="ensemble")
  model_ensemble.summary()
  model_ensemble.compile(loss="binary_crossentropy",
                          optimizer=tf.keras.optimizers.Adam(),
                          metrics=["accuracy"])

  return model_ensemble

### Plot and Curves

In [ ]:
def plot_all_images(images_list, titles):
  rows = 4
  columns = 4

  fig = plt.figure(figsize=(15, 15))

  for i in range(16):
    img = mpimg.imread(images_list[i])

    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
    plt.axis('off')
    plt.title(titles[i])

In [ ]:
def plot_accuracy_curves(historys, titles):
  fig = plt.figure(figsize=(len(historys) * 5, 3))
  rows = 1
  columns = len(historys)

  for i in range(len(historys)):
    loss = historys[i]["loss"]
    val_loss = historys[i]["val_loss"]

    accuracy = historys[i]["accuracy"]
    val_accuracy = historys[i]["val_accuracy"]

    epochs = range(len(historys[i]["loss"]))

    fig.add_subplot(rows, columns, i+1)

    plt.plot(epochs, accuracy, label="accuracy", color="lightseagreen")
    plt.plot(epochs, val_accuracy, label="val_accuracy", color="salmon")
    plt.title(titles[i])
    plt.xlabel("epochs")
    plt.legend()

In [ ]:
def plot_loss_curves(historys, titles, colors):
  fig = plt.figure(figsize=(10, 3))
  rows = 1
  columns = 2

  fig.add_subplot(rows, columns, 1)
  for i in range(len(historys)):
    accuracy = historys[i]["accuracy"]
    val_accuracy = historys[i]["val_accuracy"]

    epochs = range(len(historys[i]["loss"]))

    # Plot Accuracy
    plt.plot(epochs, accuracy, label=titles[i], color=colors[i])
    plt.title("Training Accuracy")
    plt.xlabel("epochs")
    plt.legend()


  fig.add_subplot(rows, columns, 2)
  for i in range(len(historys)):
    loss = historys[i]["loss"]
    val_loss = historys[i]["val_loss"]

    epochs = range(len(historys[i]["loss"]))

    # Plot loss
    plt.plot(epochs, loss, label=titles[i], color=colors[i])
    plt.title("Training Loss")
    plt.xlabel("epochs")
    plt.legend()

## Create and Save Models

#### Pre-trained Models

##### Original Dataset

###### Generate Data

In [ ]:
train_data, test_data, validation_data = generate_data(
                  train_dir = "/content/drive/MyDrive/larynx_dataset_original/train",
                  test_dir = "/content/drive/MyDrive/larynx_dataset_original/test",
                  valid_dir = "/content/drive/MyDrive/larynx_dataset_original/validation"
)

###### MobileNet V3

In [ ]:
mobilenet_v3 = "https://www.kaggle.com/models/google/mobilenet-v3/frameworks/TensorFlow2/variations/large-075-224-feature-vector/versions/1"

# Create Resnet model
mobilenet_v3_model = create_model(mobilenet_v3)

# Compile model
mobilenet_v3_model.compile(loss="binary_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])
# Create an empty dictionary to store the training history
mobilenet_aug_history = {'loss': [], 'accuracy': [], 'val_loss': [], 'val_accuracy': []}

# Define a callback to update the history dictionary during training
class HistoryCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        mobilenet_aug_history['loss'].append(logs['loss'])
        mobilenet_aug_history['accuracy'].append(logs['accuracy'])
        mobilenet_aug_history['val_loss'].append(logs['val_loss'])
        mobilenet_aug_history['val_accuracy'].append(logs['val_accuracy'])

history_callback = HistoryCallback()

# Fit the model
mobilenet_v3_history = mobilenet_v3_model.fit(train_data,
                                  epochs=50,
                                  steps_per_epoch=len(train_data),
                                  validation_data=validation_data,
                                  validation_steps=len(validation_data),
                                  callbacks=[history_callback, early_stopping, reduce_lr])

history_df = pd.DataFrame(mobilenet_aug_history)
history_df.to_csv('/content/drive/MyDrive/model_history/mobilenet_main_history.csv', index=False)

mobilenet_v3_model.save("/content/drive/MyDrive/models/mobilenet_main")

mobilenet_v3.evaluate(test_data)

###### EfficientNetB0 V2

In [ ]:
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

# Create EfficientNetB0 feature extractor model
efficientnet_model_main = create_model(model_url=efficientnet_url)

# Compile model
efficientnet_model_main.compile(loss="binary_crossentropy",
                           optimizer=tf.keras.optimizers.Adam(),
                           metrics=["accuracy"])

# Create an empty dictionary to store the training history
efficient_main_history = {'loss': [], 'accuracy': [], 'val_loss': [], 'val_accuracy': []}

# Define a callback to update the history dictionary during training
class HistoryCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        efficient_main_history['loss'].append(logs['loss'])
        efficient_main_history['accuracy'].append(logs['accuracy'])
        efficient_main_history['val_loss'].append(logs['val_loss'])
        efficient_main_history['val_accuracy'].append(logs['val_accuracy'])

history_callback = HistoryCallback()

# Fit model
efficientnet_history = efficientnet_model_main.fit(train_data,
                                              epochs=50,
                                              steps_per_epoch=len(train_data),
                                              validation_data=validation_data,
                                              validation_steps=len(validation_data),
                                              callbacks=[history_callback, early_stopping, reduce_lr])

history_df = pd.DataFrame(efficient_main_history)
history_df.to_csv('/content/drive/MyDrive/model_history/efficient_original_history.csv', index=False)

efficientnet_model_main.save("/content/drive/MyDrive/models/efficientnet_original")

efficientnet_model_main.evaluate(test_data)

###### ResNet50 V2

In [ ]:
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"

# Create resnet feature extractor model
resnet_model_original = create_model(model_url=resnet_url)

# Compile model
resnet_model_original.compile(loss="binary_crossentropy",
                           optimizer=tf.keras.optimizers.Adam(),
                           metrics=["accuracy"])

# Create an empty dictionary to store the training history
resnet_original_history = {'loss': [], 'accuracy': [], 'val_loss': [], 'val_accuracy': []}

# Define a callback to update the history dictionary during training
class HistoryCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        resnet_original_history['loss'].append(logs['loss'])
        resnet_original_history['accuracy'].append(logs['accuracy'])
        resnet_original_history['val_loss'].append(logs['val_loss'])
        resnet_original_history['val_accuracy'].append(logs['val_accuracy'])

history_callback = HistoryCallback()

# Fit model
resnet_history = resnet_model_original.fit(train_data,
                                              epochs=50,
                                              steps_per_epoch=len(train_data),
                                              validation_data=validation_data,
                                              validation_steps=len(validation_data),
                                              callbacks=[history_callback, early_stopping, reduce_lr])

history_df = pd.DataFrame(resnet_original_history)
history_df.to_csv('/content/drive/MyDrive/model_history/resnet_original_history.csv', index=False)

resnet_model_original.save("/content/drive/MyDrive/models/resnet_original")

resnet_model_original.evaluate(test_data)

##### Augmented Dataset

###### Generate Data

In [ ]:
train_data, test_data, validation_data = generate_data(
                  train_dir = "/content/drive/MyDrive/larynx_dataset_augmented/train",
                  test_dir = "/content/drive/MyDrive/larynx_dataset_augmented/test",
                  valid_dir = "/content/drive/MyDrive/larynx_dataset_augmented/validation"
)

###### MobileNet v3

In [ ]:
mobilenet_v3 = "https://www.kaggle.com/models/google/mobilenet-v3/frameworks/TensorFlow2/variations/large-075-224-feature-vector/versions/1"

# Create Resnet model
mobilenet_v3_model = create_model(mobilenet_v3)

# Compile model
mobilenet_v3_model.compile(loss="binary_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])
# Create an empty dictionary to store the training history
mobilenet_aug_history = {'loss': [], 'accuracy': [], 'val_loss': [], 'val_accuracy': []}

# Define a callback to update the history dictionary during training
class HistoryCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        mobilenet_aug_history['loss'].append(logs['loss'])
        mobilenet_aug_history['accuracy'].append(logs['accuracy'])
        mobilenet_aug_history['val_loss'].append(logs['val_loss'])
        mobilenet_aug_history['val_accuracy'].append(logs['val_accuracy'])

history_callback = HistoryCallback()

# Fit the model
mobilenet_v3_history = mobilenet_v3_model.fit(train_data,
                                  epochs=50,
                                  steps_per_epoch=len(train_data),
                                  validation_data=validation_data,
                                  validation_steps=len(validation_data),
                                  callbacks=[history_callback, early_stopping, reduce_lr])

history_df = pd.DataFrame(mobilenet_aug_history)
history_df.to_csv('/content/drive/MyDrive/model_history/mobilenet_augmented_history.csv', index=False)

mobilenet_v3_model.save("/content/drive/MyDrive/models/mobilenet_augmented")

mobilenet_v3.evaluate(test_data)

###### EfficientNetB0 V2

In [ ]:
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

# Create EfficientNetB0 feature extractor model
efficientnet_model_main = create_model(model_url=efficientnet_url)

# Compile model
efficientnet_model_main.compile(loss="binary_crossentropy",
                           optimizer=tf.keras.optimizers.Adam(),
                           metrics=["accuracy"])

# Create an empty dictionary to store the training history
efficient_main_history = {'loss': [], 'accuracy': [], 'val_loss': [], 'val_accuracy': []}

# Define a callback to update the history dictionary during training
class HistoryCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        efficient_main_history['loss'].append(logs['loss'])
        efficient_main_history['accuracy'].append(logs['accuracy'])
        efficient_main_history['val_loss'].append(logs['val_loss'])
        efficient_main_history['val_accuracy'].append(logs['val_accuracy'])

history_callback = HistoryCallback()

# Fit model
efficientnet_history = efficientnet_model_main.fit(train_data,
                                              epochs=50,
                                              steps_per_epoch=len(train_data),
                                              validation_data=validation_data,
                                              validation_steps=len(validation_data),
                                              callbacks=[history_callback, early_stopping, reduce_lr])

history_df = pd.DataFrame(efficient_main_history)
history_df.to_csv('/content/drive/MyDrive/model_history/efficient_augmented_history.csv', index=False)

efficientnet_model_main.save("/content/drive/MyDrive/models/efficientnet_augmented")

efficientnet_model_main.evaluate(test_data)

###### ResNet50 V2

In [ ]:
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"

# Create resnet feature extractor model
resnet_model_augmented = create_model(model_url=resnet_url)

# Compile model
resnet_model_augmented.compile(loss="binary_crossentropy",
                           optimizer=tf.keras.optimizers.Adam(),
                           metrics=["accuracy"])

# Create an empty dictionary to store the training history
resnet_augmented_history = {'loss': [], 'accuracy': [], 'val_loss': [], 'val_accuracy': []}

# Define a callback to update the history dictionary during training
class HistoryCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        resnet_augmented_history['loss'].append(logs['loss'])
        resnet_augmented_history['accuracy'].append(logs['accuracy'])
        resnet_augmented_history['val_loss'].append(logs['val_loss'])
        resnet_augmented_history['val_accuracy'].append(logs['val_accuracy'])

history_callback = HistoryCallback()

# Fit model
resnet_history = resnet_model_augmented.fit(train_data,
                                              epochs=50,
                                              steps_per_epoch=len(train_data),
                                              validation_data=validation_data,
                                              validation_steps=len(validation_data),
                                              callbacks=[history_callback, early_stopping, reduce_lr])

history_df = pd.DataFrame(resnet_augmented_history)
history_df.to_csv('/content/drive/MyDrive/model_history/resnet_augmented_history.csv', index=False)

resnet_model_augmented.save("/content/drive/MyDrive/models/resnet_augmented")

resnet_model_augmented.evaluate(test_data)

#### Ensemble Models

###### Original Dataset

In [ ]:
mobilenet_model = keras.models.load_model("/content/drive/MyDrive/models/mobilenet_orginal")
efficientnet_model = keras.models.load_model("/content/drive/MyDrive/models/efficientnet_original")
resnet_loaded_model = keras.models.load_model("/content/drive/MyDrive/models/resnet_original")

models = []

models.append(mobilenet_model)
models.append(efficientnet_model)
models.append(resnet_loaded_model)

model_input = layers.Input(shape=models[0].input_shape[1:])

ensemble_model = ensemble(models, model_input)

scores = ensemble_model.evaluate(test_data,
                                  steps=len(test_data))
print("Mobile_v3 + Efficient_B0 + ResNet_v2 Accuracy = ", scores[1])

###### Augmented Dataset

In [ ]:
mobilenet_model = keras.models.load_model("/content/drive/MyDrive/models/mobilenet_augmented")
efficientnet_model = keras.models.load_model("/content/drive/MyDrive/models/efficientnet_augmented")
resnet_loaded_model = keras.models.load_model("/content/drive/MyDrive/models/resnet_augmented")

models = []

models.append(mobilenet_model)
models.append(efficientnet_model)
models.append(resnet_loaded_model)

model_input = layers.Input(shape=models[0].input_shape[1:])

ensemble_model = ensemble(models, model_input)

scores = ensemble_model.evaluate(test_data,
                                  steps=len(test_data))
print("Mobile_v3 + Efficient_B0 + ResNet_v2 Accuracy = ", scores[1])

#### Stack Models

###### Original Dataset

In [ ]:
# Load pre-trained model
mobilenet_model = tf.keras.models.load_model("/content/drive/MyDrive/models/mobilenet_original")
efficientnet_model = tf.keras.models.load_model("/content/drive/MyDrive/models/efficientnet_original")
resnet_loaded_model = tf.keras.models.load_model("/content/drive/MyDrive/models/resnet_original")

models = [mobilenet_model, efficientnet_model, resnet_loaded_model]

# Freeze the layers of base models
for model in models:
    model.trainable = False

models[0]._name = 'mobilenet_model_original'
models[1]._name = 'efficientnet_model_original'
models[2]._name = 'resnet_model_original'

for i, model in enumerate(models):
    for layer in model.layers:
        layer._name = f"{model.name}_layer_{layer.name}"

# Create a stacking model
model_input = layers.Input(shape=(224, 224, 3))  # Input shape should match the input shape of your images
mobilenet_output = mobilenet_model(model_input)
efficientnet_output = efficientnet_model(model_input)
resnet_output = resnet_loaded_model(model_input)

# Concatenate the outputs of the base models
concatenated = layers.Concatenate(axis=-1)([mobilenet_output, efficientnet_output, resnet_output])

# Add a dense layer for the meta-model
stacked_model_output = layers.Dense(1, activation='sigmoid')(concatenated)

# Create the stacked model
stacked_model = tf.keras.models.Model(inputs=model_input, outputs=stacked_model_output, name="stacked_model")

stacked_model.compile(loss="binary_crossentropy",
                      optimizer=tf.keras.optimizers.Adam(),
                      metrics=["accuracy"])

stacked_model.summary()

# Create an empty dictionary to store the training history
stacked_original_history = {'loss': [], 'accuracy': [], 'val_loss': [], 'val_accuracy': []}

# Define a callback to update the history dictionary during training
class HistoryCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        stacked_original_history['loss'].append(logs['loss'])
        stacked_original_history['accuracy'].append(logs['accuracy'])
        stacked_original_history['val_loss'].append(logs['val_loss'])
        stacked_original_history['val_accuracy'].append(logs['val_accuracy'])

history_callback = HistoryCallback()

# Train the stacked model
stacked_model.fit(train_data,
                  epochs=50,  # Adjust the number of epochs based on your requirements
                  validation_data=validation_data,
                  steps_per_epoch=len(train_data),
                  validation_steps=len(validation_data),
                  callbacks=[history_callback, early_stopping, reduce_lr])

history_df = pd.DataFrame(stacked_original_history)
history_df.to_csv('/content/drive/MyDrive/model_history/stacked_original_history.csv', index=False)

stacked_model.save("/content/drive/MyDrive/models/stacked_original")

scores = stacked_model.evaluate(test_data, steps=len(test_data))
print("Stacked Model Accuracy = ", scores[1])

##### Augmented Dataset

In [ ]:
# Load pre-trained model
mobilenet_model = tf.keras.models.load_model("/content/drive/MyDrive/models/mobilenet_augmented")
efficientnet_model = tf.keras.models.load_model("/content/drive/MyDrive/models/efficientnet_augmented")
resnet_loaded_model = tf.keras.models.load_model("/content/drive/MyDrive/models/resnet_augmented")

models = [mobilenet_model, efficientnet_model, resnet_loaded_model]

# Freeze the layers of base models
for model in models:
    model.trainable = False

models[0]._name = 'mobilenet_model_augmented'
models[1]._name = 'efficientnet_model_augmented'
models[2]._name = 'resnet_model_augmented'

for i, model in enumerate(models):
    for layer in model.layers:
        layer._name = f"{model.name}_layer_{layer.name}"

# Create a stacking model
model_input = layers.Input(shape=(224, 224, 3))  # Input shape should match the input shape of your images
mobilenet_output = mobilenet_model(model_input)
efficientnet_output = efficientnet_model(model_input)
resnet_output = resnet_loaded_model(model_input)

# Concatenate the outputs of the base models
concatenated = layers.Concatenate(axis=-1)([mobilenet_output, efficientnet_output, resnet_output])

# Add a dense layer for the meta-model
stacked_model_output = layers.Dense(1, activation='sigmoid')(concatenated)

# Create the stacked model
stacked_model = tf.keras.models.Model(inputs=model_input, outputs=stacked_model_output, name="stacked_model")

stacked_model.compile(loss="binary_crossentropy",
                      optimizer=tf.keras.optimizers.Adam(),
                      metrics=["accuracy"])

stacked_model.summary()

# Create an empty dictionary to store the training history
stacked_augmented_history = {'loss': [], 'accuracy': [], 'val_loss': [], 'val_accuracy': []}

# Define a callback to update the history dictionary during training
class HistoryCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        stacked_augmented_history['loss'].append(logs['loss'])
        stacked_augmented_history['accuracy'].append(logs['accuracy'])
        stacked_augmented_history['val_loss'].append(logs['val_loss'])
        stacked_augmented_history['val_accuracy'].append(logs['val_accuracy'])

history_callback = HistoryCallback()

# Train the stacked model
stacked_model.fit(train_data,
                  epochs=50,  # Adjust the number of epochs based on your requirements
                  validation_data=validation_data,
                  steps_per_epoch=len(train_data),
                  validation_steps=len(validation_data),
                  callbacks=[history_callback, early_stopping, reduce_lr])

history_df = pd.DataFrame(stacked_augmented_history)
history_df.to_csv('/content/drive/MyDrive/model_history/stacked_augmented_history.csv', index=False)

stacked_model.save("/content/drive/MyDrive/models/stacked_augmented")

scores = stacked_model.evaluate(test_data, steps=len(test_data))
print("Stacked Model Accuracy = ", scores[1])

## Create Plots

#### Plot images of all subclasses alphabetically

In [ ]:
img_list = [
    '/content/drive/MyDrive/438_plot_images/Amyloidosis/P056 (2).jpg',
    '/content/drive/MyDrive/438_plot_images/Carcinoma_in_situ/P013 (38).jpg',
    '/content/drive/MyDrive/438_plot_images/Cyst/P037 (42).jpg',
    '/content/drive/MyDrive/438_plot_images/Granuloma/P167 (5).jpg',
    '/content/drive/MyDrive/438_plot_images/Hemangioma/P123 (42).jpg',
    '/content/drive/MyDrive/438_plot_images/High_grade_dyspasia/P142 (4).jpg',
    '/content/drive/MyDrive/438_plot_images/Hyperkeratosis/P032 (20).jpg',
    '/content/drive/MyDrive/438_plot_images/Hyperplasia/P053 (6).jpg',
    '/content/drive/MyDrive/438_plot_images/Inflammation/P090 (5).jpg',
    '/content/drive/MyDrive/438_plot_images/Low_grade_dyspasia/P129 (16).jpg',
    '/content/drive/MyDrive/438_plot_images/Namboo_node/P208 (15).jpg',
    '/content/drive/MyDrive/438_plot_images/Nodule/P049 (22).jpg',
    '/content/drive/MyDrive/438_plot_images/Papillomatosis/P133 (26).jpg',
    '/content/drive/MyDrive/438_plot_images/Polyp/P038 (29).jpg',
    '/content/drive/MyDrive/438_plot_images/Reinkes_edema/P029 (15).jpg',
    '/content/drive/MyDrive/438_plot_images/SCC/P001 (5).jpg',
]

titles = [
    "Amyloidosis",
    "Carcinoma in situ",
    "Cyst",
    "Granuloma",
    "Hemangioma",
    "High grade dyspasia",
    "Hyperkeratosis",
    "Hyperplasia",
    "Inflammation",
    "Low grade dyspasia",
    "Namboo node",
    "Nodule",
    "Papillomatosis",
    "Polyp",
    "Renkes edema",
    "SCC",
]

plot_all_images(img_list, titles)

#### Compare Biased Classes before and after augmentation

In [ ]:
X = ['Raw Dataset','Augmented Dataset']
Benign = [7657, 7657]
Malignant = [3487, 692+4872+1384]

X_axis = np.arange(len(X))

plt.bar(X_axis - 0.2, Benign, 0.4, label = 'Benign', color="salmon")
plt.bar(X_axis + 0.2, Malignant, 0.4, label = 'Malignant', color="lightseagreen")

plt.xticks(X_axis, X)
plt.ylim(0, 10000)
plt.xlabel("Type")
plt.ylabel("Number of Images")
plt.legend()
plt.show()

#### Plot Accuracy Curves of Stacked Models

In [ ]:
stacked_aug = pd.read_csv("/content/drive/MyDrive/model_history/stacked_aug_history.csv")
stacked_main = pd.read_csv("/content/drive/MyDrive/model_history/stacked_main_history.csv")

historys = []
historys.append(stacked_aug)
historys.append(stacked_main)

plot_accuracy_curves(history, ["Stacked Aug", "Stacked Raw"])

#### Plot Accuracy Curves of Base Models on Augmented Dataset

In [ ]:
historys = []

mobile_net = pd.read_csv("/content/drive/MyDrive/model_history/mobilenet_aug_history.csv")
efficient_net = pd.read_csv("/content/drive/MyDrive/model_history/efficient_aug_history.csv")
res_net = pd.read_csv("/content/drive/MyDrive/model_history/resnet_aug.csv")

historys.append(mobile_net)
historys.append(efficient_net)
historys.append(res_net)

plot_accuracy_curves(historys, ["MobileNet V3", "EfficientNetB0 V2", "ResNet50 V2"])

#### Plot Accuracy Curves of Base Models on Original Dataset

In [ ]:
historys = []

mobile_net = pd.read_csv("/content/drive/MyDrive/model_history/mobilenet_main_history.csv")
efficient_net = pd.read_csv("/content/drive/MyDrive/model_history/efficient_main_history.csv")
res_net = pd.read_csv("/content/drive/MyDrive/model_history/resnet_main.csv")

historys.append(mobile_net)
historys.append(efficient_net)
historys.append(res_net)

plot_accuracy_curves(historys, ["MobileNet V3", "EfficientNetB0 V2", "ResNet50 V2"])

#### Plot Accuracy and Loss Curves of all Models

In [ ]:
historys = []

mobile_net = pd.read_csv("/content/drive/MyDrive/model_history/mobilenet_aug_history.csv")
efficient_net = pd.read_csv("/content/drive/MyDrive/model_history/efficient_aug_history.csv")
res_net = pd.read_csv("/content/drive/MyDrive/model_history/resnet_aug.csv")
stacked_aug = pd.read_csv("/content/drive/MyDrive/model_history/stacked_aug_history.csv")
stacked_main = pd.read_csv("/content/drive/MyDrive/model_history/stacked_main_history.csv")

titles = ["MobileNet", "EfficientNet", "ResNet", "Stacked", "Stacked_aug"]
colors = ["orchid", "orange", "mediumslateblue", "mediumseagreen", "salmon"]

historys.append(mobile_net)
historys.append(efficient_net)
historys.append(res_net)
historys.append(stacked_main)
historys.append(stacked_aug)

plot_loss_curves(historys, titles, colors)